In [1]:
# local python tests
from pprint import pprint
from igraph import *
import igraph.test
import pickle
import os
import numpy as np
from scipy.sparse import csr_matrix
import collections
import copy




"""

    %%%%%%%%%%%%%%%%%%%%%%%%%%% gcn tests %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    x  feature vectors
    tx feature vectors for test (unlabeled)
    allx feature vectors for all

    y  label vectors       numpy.ndarray
    ty test label vectors  numpy.ndarray
    ally  test label of all

    graph : adjacency matrix (dict of lists)
    test.index: node id list for tests?


    steps
    1) inspect ind.citeseer.x/tx/y/ty/allx/ally/test.index

    2) build those files from code

    3) call learning algorithm
        cd src/notebooks
        python gcn_tests_input.py&&  cp ind.* ../gcn/gcn/gcn/data
        cd src/gcn/gcn/gcn
        python train.py --dataset t1


    4) look at embedding result

"""


def pickleToDisk(myobj, filepath):
    pickle.dump(myobj, open(filepath, "wb"))


def printGraphObject(filepath):


    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    y = pickle.load(open(filepath,'rb'))
    print(type(y))
    if type(y) ==  type([]) or type(y) == type(np.ndarray([])):
        
        print(len(y))
        pprint(y[:min(3,len(y)-1)])
    elif type(y) == type({}) or type(y)==type(collections.defaultdict()):
        pprint(y.keys()[:min(3,len(y)-1)])
        pprint(y[y.keys()[0]])
        pprint(y[y.keys()[1]])
        pprint(y[y.keys()[20]])
        pprint(y[y.keys()[3000]])

    else:
        
        pprint(y.shape)
        pprint(y[0,:min(3,y.shape[1]-1)].toarray())
        pprint(y[0,0].shape)
        pprint(type(y[0,0]))
        pprint(y[0,0])

def readListFile(filepath):
    myobject = os.path.basename(filepath)
    print("\n\n*************** "+myobject+" ********************")
    

    testInstances =[]
    with open(filepath, 'rb') as f:
        for line in f.readlines():
            testInstances.append(int(line))

    pprint(testInstances[:min(3,len(testInstances)-1)])


def inspectData():

    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.y')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ty')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.x')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.tx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.graph')
    readListFile('../gcn/gcn/gcn/data/ind.citeseer.test.index')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.allx')
    printGraphObject('../gcn/gcn/gcn/data/ind.citeseer.ally')





def generateData(suffix):
    
    

    #igraph.test.run_tests()

    # generation
    #g = Graph.Tree(127, 2)
    #g = Graph.GRG(100, 0.2)
    #g = Graph.Erdos_Renyi(100, 0.2)
    #g = Graph.Watts_Strogatz(1, 100, 4, 0.5 )
    n_node = 1700
    g = Graph.Barabasi(n_node)
    summary(g)

    # graph metrics
    # pprint(g.degree([2,3,6,99]))
    # pprint(g.edge_betweenness())
    # pprint(g.pagerank())
    #pprint(g.get_adjacency())
    #pprint(dir(g))
    


    # test.index
    # list first 5 nodes -> to test.index
    n_train = 100
    n_test = 300
    testList = range(n_node-n_test, n_node)
    with open(suffix+'.test.index','wb') as f:
        for i in testList:
            f.write(str(i)+"\n")

    # adjacency dict (graph)
    i = 0
    A = g.get_adjlist()
    graphAdj = {}
    for node in A:
        graphAdj[i] = node
        i+=1
    pickleToDisk(graphAdj, suffix+".graph")

    # features (allx, x, tx)
    #  no features -> a 0 for each node 
    #   or node degree
    #  into a sparce matrix
    xFeatures = g.degree()
    #xFeatures = [feature * 1.0 for feature in xFeatures]
    xFeatures = np.array(xFeatures, np.float64)
    print(xFeatures[:10])
    # see if any degree is negative
    degs = np.array(xFeatures)
    degs2 = degs < 0
    degs3 = degs2.sum(axis=0)
    print(degs3)
    allx = copy.deepcopy(xFeatures[range(n_node-n_test)])
    x = copy.deepcopy(xFeatures[range(n_train)])
    #tx = copy.deepcopy(xFeatures[range(n_node-n_test, n_node)])
    tx = [xFeatures[i] for i in testList]

    """
    # remove test List
    # remove also validation
    #for elem in tx:
    #    x.pop(x.index(elem))
    for elindex in testList:
        x.pop(elindex)
    
    validationList = range(testList[-1]+1, testList[-1]+501)
    for elindex in validationList:
        x.pop(elindex)
    """
    
    allx = csr_matrix(allx)
    allx = np.transpose(allx)
    pickleToDisk(allx, suffix+".allx")
    x = csr_matrix(x)
    x = np.transpose(x)
    pickleToDisk(x, suffix+".x")
    tx = csr_matrix(tx)
    tx = np.transpose(tx)
    pickleToDisk(tx, suffix+".tx")

    print(x.shape)
    print(tx.shape)
    print(allx.shape)
    


    # labels (ally,y , ty)
    prs = g.pagerank()
    print(prs[:10])
    prs = np.array(prs)
    threshold = 0.01
    prs = prs >= threshold
    prs2 = prs < threshold

    prs = np.vstack((prs,prs2)).T
    pprint(prs)

    prs_all = copy.deepcopy(prs[range(n_node-n_test)])
    prs_train = copy.deepcopy(prs[range(n_train)])
    prs_test = [prs[i] for i in testList]
    #testList.extend(validationList)
    #finalList = testList
    #prs_train = [ prs[i] for i in range(prs.shape[0]) if i not in finalList ]

    ally = np.array(prs_all)
    y = np.array(prs_train)  
    ty = np.array(prs_test)


    pickleToDisk(ally, suffix+".ally")
    pickleToDisk(y, suffix+".y")
    pickleToDisk(ty, suffix+".ty")
    
    print(y.shape)
    print(ty.shape)
    print(ally.shape)
    
    

    # run training on those files

In [2]:
generateData("ind.t2")

IGRAPH U--- 1700 1699 -- 
[106.   1.   1.  29.  56.  21.   1.  12.  17.   2.]
0
(100, 1)
(300, 1)
(1400, 1)
[0.02608542064890646, 0.00029741083705712155, 0.00029741083705712155, 0.007147725922553707, 0.013851332276579196, 0.005114423116241617, 0.00029524765834657674, 0.0030641172827451124, 0.004225016638849887, 0.0005847025134382265]
array([[ True, False],
       [False,  True],
       [False,  True],
       ...,
       [False,  True],
       [False,  True],
       [False,  True]])
(100, 2)
(300, 2)
(1400, 2)
